# Multilayer Perceptron 

In [0]:
import tensorflow as tf


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [0]:
# Parameters
learning_rate= 0.001
training_epochs = 50
batch_size = 10
display_step = 1
# Network parameter
width  =28
height = 28
flat = height*width
n_input= height*width
n_hidden_1 = 256
n_hidden_2= 256
n_classes = 10
# Input and output
X = tf.placeholder(tf.float32, shape=[None, n_input])
Y = tf.placeholder(tf.float32, shape=[None, n_classes])


In [0]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases={
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_classes])) 
}

In [0]:
# Create model
def multilayer_perceptron(x):
  layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
  layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
  out_layer = tf.matmul(layer_2, weights['out'])+biases['out']
  return out_layer

In [24]:
# Construct model
logits = multilayer_perceptron(X)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Initializing the variables
init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)
  for epoch in range(training_epochs):
    avg_cost= 0
    total_batch= int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
      batch_x, batch_y = mnist.train.next_batch(batch_size)
      _, c = sess.run([train_op, loss_op], feed_dict={X:batch_x, Y: batch_y})
      avg_cost +=c/total_batch
    if epoch%display_step==0:
      print("Epoch:", "%04d"%(epoch+1), "cost={:.9f}".format(avg_cost))
  print("Optimization Finished!")
# Test model
  pred = tf.nn.softmax(logits)
  correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))

# Calculate accurary
  accurary = tf.reduce_mean(tf.cast(correct_prediction, "float"))
  print("Accurary:", accurary.eval({X:mnist.test.images, Y:mnist.test.labels}))

Epoch: 0001 cost=165.057807488
Epoch: 0002 cost=78.562322542
Epoch: 0003 cost=58.739451151
Epoch: 0004 cost=47.489713923
Epoch: 0005 cost=40.015347818
Epoch: 0006 cost=34.379450563
Epoch: 0007 cost=30.447749995
Epoch: 0008 cost=27.136583856
Epoch: 0009 cost=24.653714675
Epoch: 0010 cost=22.863608650
Epoch: 0011 cost=21.293963007
Epoch: 0012 cost=20.052441568
Epoch: 0013 cost=18.776118606
Epoch: 0014 cost=17.646182039
Epoch: 0015 cost=16.720625100
Epoch: 0016 cost=15.847718932
Epoch: 0017 cost=14.854909516
Epoch: 0018 cost=14.085203423
Epoch: 0019 cost=13.448182639
Epoch: 0020 cost=12.814833759
Epoch: 0021 cost=12.276647021
Epoch: 0022 cost=11.475711801
Epoch: 0023 cost=11.061195459
Epoch: 0024 cost=10.415391955
Epoch: 0025 cost=10.063829785
Epoch: 0026 cost=9.515111806
Epoch: 0027 cost=9.115835304
Epoch: 0028 cost=8.634803654
Epoch: 0029 cost=8.199377259
Epoch: 0030 cost=7.838775187
Epoch: 0031 cost=7.437354064
Epoch: 0032 cost=7.176634787
Epoch: 0033 cost=6.754681740
Epoch: 0034 cost=

In [68]:
from google.colab import files
files.upload()

Saving testmnist.csv to testmnist.csv


# Convolution Neural Network

In [25]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("tmp/data", one_hot=True)


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
# Training Parameters
learning_rate = 0.003
num_steps= 1000
batch_size=128
display_step=10

# Network Parameters
num_input= 784
num_classes = 10
dropout= 0.75

X= tf.placeholder(tf.float32, [None, num_input])
Y= tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)


In [0]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [67]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training# Start 
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")
    
    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0}))

Step 1, Minibatch Loss= 57075.8359, Training Accuracy= 0.148
Step 10, Minibatch Loss= 4868.1460, Training Accuracy= 0.672
Step 20, Minibatch Loss= 4017.6157, Training Accuracy= 0.836
Step 30, Minibatch Loss= 2215.9648, Training Accuracy= 0.883
Step 40, Minibatch Loss= 1818.1929, Training Accuracy= 0.906
Step 50, Minibatch Loss= 1968.1517, Training Accuracy= 0.898
Step 60, Minibatch Loss= 1143.1051, Training Accuracy= 0.891
Step 70, Minibatch Loss= 1316.0112, Training Accuracy= 0.914
Step 80, Minibatch Loss= 1435.0190, Training Accuracy= 0.922
Step 90, Minibatch Loss= 1125.3888, Training Accuracy= 0.930
Step 100, Minibatch Loss= 714.3336, Training Accuracy= 0.906
Step 110, Minibatch Loss= 854.2839, Training Accuracy= 0.930
Step 120, Minibatch Loss= 985.0384, Training Accuracy= 0.891
Step 130, Minibatch Loss= 319.9572, Training Accuracy= 0.961
Step 140, Minibatch Loss= 211.2625, Training Accuracy= 0.969
Step 150, Minibatch Loss= 888.2336, Training Accuracy= 0.906
Step 160, Minibatch Loss

# Recurrent Neural Network

In [85]:

import tensorflow as tf
from tensorflow.contrib import rnn
tf.reset_default_graph()
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:

# Training Parameters# Train 
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [0]:
# Define weights
weights={
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases={
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [0]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']


In [89]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

with tf.Session() as sess:
  # Run the initializer
  sess.run(init)
  for step in range(1, training_steps+1):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    
    batch_x = batch_x.reshape((batch_size, timesteps, num_input))
    # Run optimization op(backprop)
    sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
    if(step%display_step==0 or step==1):
      loss, acc = sess.run([loss_op, accurary], feed_dict={X:batch_x, Y:batch_y})
      print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
  print("Optimization Finished!")

  # Calculate accuracy for 128 mnist test images
  test_len = 128
  test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
  test_label = mnist.test.labels[:test_len]
  print("Testing Accuracy:", \
    sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))
                       

ValueError: ignored

# Auto-Encoder

In [90]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
# Training Parameters
learning_rate=0.01
num_steps= 3000
display_step=1000
examples_to_show=10

# Network Parameters
num_hidden_1=256
num_hidden_2=128
num_input=784

#tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])

weights={
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
}

biases={
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
}

In [96]:
def encoder(x):
  layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
  layer_2= tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
  return layer_2
def decoder(x):
  layer_1= tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
  layer_2= tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
  return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

y_pred = decoder_op
y_true=X
loss = tf.reduce_mean(tf.pow(y_true-y_pred, 2))
optimizer= tf.train.RMSPropOptimizer(learning_rate).minimize(loss)
init = tf.global_variables_initializer()
# Start Training 
# Start a new TF session
sess = tf.Session()
sess.run(init)
for i in range(1, num_steps+1):
  batch_x, _ = mnist.train.next_batch(batch_size)
  _, l = sess.run([optimizer, loss], feed_dict={X: batch_x})
  
  if(i%display_step==0 or i==1):
    print("Step %i: Minibatch Loss: %f"%(i, l))
  

  

Step 1: Minibatch Loss: 0.465861
Step 1000: Minibatch Loss: 0.134085
Step 2000: Minibatch Loss: 0.109898
Step 3000: Minibatch Loss: 0.099270


# Variational Auto-Encoder

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import tensorflow as tf


In [98]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
# Parameters
learning_rate = 0.001
num_steps = 30000
batch_size=64

# Network Parameters
image_dim= 784
hidden_dim = 512
latent_dim= 2
def glorot_init(shape):
  return tf.random_normal(shape=shape, stddev=1./tf.sqrt(shape[0]/2.))


In [100]:
# Variables
weights={
    'encoder_h1': tf.Variable(glorot_init([image_dim, hidden_dim])),
    'z_mean': tf.Variable(glorot_init([hidden_dim, latent_dim])),
    'z_std': tf.Variable(glorot_init([hidden_dim, latent_dim])),
    'decoder_h1': tf.Variable(glorot_init([latent_dim, hidden_dim])),
    'decoder_out': tf.Variable(glorot_inti([hidden_dim, image_dim]))
}
biases={
    'encoder_b1': tf.Variable(glorot_init([hidden_dim])),
    'z_mean': tf.Varialbe(glorot_init([latent_dim])),
    'z_std': tf.Varialbe(glorot_init(latent_dim)),
    'decoder_b1': tf.Variable(glorot_init([hidden_dim])),
    'decoder_out': tf.Variable(glorot_init([image_dim]))
}

SyntaxError: ignored